In [28]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold


In [29]:
train_in = np.loadtxt('./Datasets/train_in.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_in = np.loadtxt('./Datasets/test_in.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       
train_out = np.loadtxt('./Datasets/train_out.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_out = np.loadtxt('./Datasets/test_out.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       

In [30]:
X_train, X_val, y_train, y_val = train_test_split(train_in, train_out, train_size=0.8, random_state=0)
X_test = test_in
y_test = test_out
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print('Inputs_train', )

(1365, 256) (342, 256) (1365,) (342,)
Inputs_train


In [31]:
# input dataset
def preprocessing(datasets):
    return np.hstack((datasets, np.ones(len(datasets)).reshape(len(datasets),1)))

In [32]:
def get_dummies(list, array):
    for i, each in zip(list, array):
        each[int(i)] = 1
    return array

In [33]:
# sigmoid function 
def sig(x):
    return (1/(1+np.exp(-x)))

def sig_prime(x):
    return (x * (1 - x))

In [34]:
Len_train = len(X_train)
Len_val = len(X_val)
Len_test = len(X_test)

X_train = preprocessing(X_train)
X_val = preprocessing(X_val)
X_test = preprocessing(X_test)

y_train_dummy = np.zeros(Len_train*10).reshape(Len_train, 10)
y_train_dummy = get_dummies(y_train, y_train_dummy)
y_val_dummy = np.zeros(Len_val*10).reshape(Len_val, 10)
y_val_dummy = get_dummies(y_val, y_val_dummy)
y_test_dummy = np.zeros(Len_test*10).reshape(Len_test, 10)
y_test_dummy = get_dummies(y_test, y_test_dummy)

In [35]:
kf = KFold(n_splits=5)

kfold_index_list = []

for cv_index_train, cv_index_test in kf.split(X_train):
    kfold_index_list.append((cv_index_train, cv_index_test))

In [71]:
def training(lr, w, x, y_dummy):
    for iter in range(1000):
        for i in range(len(x)):
            y_hat = sig(np.dot(x[i], w))


            
            diff = y_dummy[i] - y_hat
            x_re = x[i].reshape(257,1)
            # print(y_hat)
            sp = sig_prime(y_hat) * diff
            w += lr * x_re * sp

        if iter % 10 == 0 :
            arg_y_hat = np.argmax(sig(np.dot(x, w)), axis=1)
            arg_y_dummy = np.argmax(y_dummy, axis=1)
            arg_diff = abs(arg_y_dummy - arg_y_hat)
            acc = (len(y_dummy)-np.sum(arg_diff>0))/len(y_dummy)
            print(sum(arg_diff))
        if acc > 0.97:
            break
    return w

In [58]:
w0 = np.random.normal(loc=0.0, scale=1.0, size=(257, 10))

In [72]:
# np.random.seed(2022)
# w0 = np.random.normal(1,-1,loc=0.0, scale=1.0, size=(257, 10))
# lr = 0.3
# w = training(lr, w0, X_train, y_train_dummy)

In [39]:
def predict(lr, w, train_x, y_train_dummy):
    y_hat = sig(np.dot(train_x, w))
    return np.argmax(y_hat, axis=1)

In [41]:
W_experiments = np.zeros(257*10*4).reshape(4, 257,10)


In [42]:
def generate_w0(sections):
    np.random.seed(2022)
    W_experiments = np.zeros(257*10*len(sections)).reshape(len(sections), 257,10)
    index = 0
    for i in sections:
        # print(i[0], i[1])
        W_experiments[index] = np.random.uniform(i[0], i[1], size=(257,10))
        index += 1
    return W_experiments

In [43]:
# np.random.seed(2022)
# w0 = np.random.uniform(-0.3,0.3,size=(257,10))
sections = [(0, 1), (-0.5, 0.5), (-0.3, 0.3), (-0.1, 0.1)]
lr = 0.3
W_experiments = generate_w0(sections)


In [44]:
def generate_acc(w):
    pred_val = predict(lr, w, X_val, y_val_dummy)
    acc_val = accuracy_score(y_val, pred_val)

    pred_test = predict(lr, w, X_test, y_test_dummy)
    acc_test = accuracy_score(y_test, pred_test)

    return acc_val, acc_test

In [70]:
w_list = []
results = []
w_list.clear()
results.clear()
np.random.seed(2022)
w0 = np.random.normal(loc=0.0, scale=0.3, size=(257, 10))
w = training(lr, w0, X_train, y_train_dummy)

pred_train = predict(lr, w, X_train, y_train_dummy)
acc_train = accuracy_score(y_train, pred_train)
acc_val, acc_test = generate_acc(w)

w_list.append(w)

print("Training with weights section: ", 'Completely random')
print("Accuracy train: ", acc_train, "Accuracy validation", acc_val, "Accuracy test", acc_test)

1405
782
535
542
548
544
568
549
545
544
528
525
525
529
536
125
116
114
109
105
103
103
100
101
100
100
100
100
100
105
105
105
105
105
105
105
105
105
101
97
97
97
97
97
97
99
99
99
99
99
99
99
99
99
99
97
97
97
97
97
97
97
97
97
97
97
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
Training with weights section:  Completely random
Accuracy train:  0.9758241758241758 Accuracy validation 0.956140350877193 Accuracy test 0.887


In [46]:
for w0_index in range(len(sections)):
    print(w0_index)
    w0 = W_experiments[w0_index]
    w = training(lr, w0, X_train, y_train_dummy)
    
    pred_train = predict(lr, w, X_train, y_train_dummy)
    acc_train = accuracy_score(y_train, pred_train)
    acc_val, acc_test = generate_acc(w)

    w_list.append(w)
    results.append((acc_train, acc_val, acc_test))
for i in range(len(sections)):
    acc, acc_val, acc_test = results[i]
    print("Training with weights section: ", sections[i])
    print("Accuracy train: ", acc, "Accuracy validation", acc_val, "Accuracy test", acc_test)



0
1
2
3
Training with weights section:  (0, 1)
Accuracy train:  0.189010989010989 Accuracy validation 0.21637426900584794 Accuracy test 0.229
Training with weights section:  (-0.5, 0.5)
Accuracy train:  0.9736263736263736 Accuracy validation 0.956140350877193 Accuracy test 0.887
Training with weights section:  (-0.3, 0.3)
Accuracy train:  0.9706959706959707 Accuracy validation 0.9590643274853801 Accuracy test 0.888
Training with weights section:  (-0.1, 0.1)
Accuracy train:  0.9714285714285714 Accuracy validation 0.956140350877193 Accuracy test 0.883


In [47]:
w0 = np.zeros(257*10).reshape(257, 10)
w = training(lr, w0, X_train, y_train_dummy)

pred_train = predict(lr, w, X_train, y_train_dummy)
acc_train = accuracy_score(y_train, pred_train)
acc_val, acc_test = generate_acc(w)

w_list.append(w)

print("Training with weights section: ", '0, 0')
print("Accuracy train: ", acc_train, "Accuracy validation", acc_val, "Accuracy test", acc_test)

Training with weights section:  0, 0
Accuracy train:  0.9714285714285714 Accuracy validation 0.9619883040935673 Accuracy test 0.876


In [48]:
len(w_list)

6

In [49]:
def best_weight():
    return 0

In [50]:
np.random.seed(2022)
w0 = np.random.uniform(-0.3, 0.3, size=(257,10))
lr = 0.3
w_list.clear()
results.clear()
for i in range(5):
    w = training(lr, w0, X_train[kfold_index_list[i][0]], y_train_dummy[kfold_index_list[i][0]])

    pred_train = predict(lr, w, X_train[kfold_index_list[i][1]], y_train_dummy[kfold_index_list[i][1]])
    acc_train = accuracy_score(y_train[kfold_index_list[i][1]], pred_train)

    pred_val = predict(lr, w, X_val, y_val_dummy)
    acc_val = accuracy_score(y_val, pred_val)

    pred_test = predict(lr, w, X_test, y_test_dummy)
    acc_test = accuracy_score(y_test, pred_test)

    w_list.append(w)
    results.append((acc_train, acc_val, acc_test))

for acc, acc_val, acc_test in results:
    print("Accuracy train: ", acc, "Accuracy validation", acc_val, "Accuracy test", acc_test)


Accuracy train:  0.9413919413919414 Accuracy validation 0.956140350877193 Accuracy test 0.873
Accuracy train:  0.9633699633699634 Accuracy validation 0.956140350877193 Accuracy test 0.881
Accuracy train:  0.9743589743589743 Accuracy validation 0.956140350877193 Accuracy test 0.88
Accuracy train:  0.9853479853479854 Accuracy validation 0.9590643274853801 Accuracy test 0.877
Accuracy train:  0.9853479853479854 Accuracy validation 0.9444444444444444 Accuracy test 0.874


In [51]:
len(w_list)

5

In [52]:
## confusion Test
np.random.seed(2022)
w0 = np.random.uniform(-0.3, 0.3, size=(257,10))
w = training(lr, w0, X_train, y_train_dummy)
confusion_matrix(predict(lr, w, X_test, y_test_dummy), y_test)

array([[215,   0,   3,   2,   2,   3,   2,   0,   3,   0],
       [  0, 113,   0,   0,   1,   0,   0,   0,   2,   1],
       [  1,   0,  80,   2,   0,   0,   0,   1,   3,   1],
       [  1,   0,   2,  68,   0,   9,   1,   1,   4,   0],
       [  3,   2,   6,   0,  76,   1,   1,   3,   2,   3],
       [  1,   0,   0,   1,   1,  37,   0,   0,   2,   2],
       [  1,   3,   2,   0,   2,   1,  86,   0,   0,   0],
       [  0,   2,   2,   2,   1,   2,   0,  56,   4,   1],
       [  1,   0,   6,   3,   0,   0,   0,   0,  72,   1],
       [  1,   1,   0,   1,   3,   2,   0,   3,   0,  79]], dtype=int64)

In [53]:
# w0 = np.random.random((257,10))
# lr = 0.3
# w = training(lr, w0, X_train, y_train_dummy)

In [121]:
test_in_sort = np.hstack((test_in, test_out.reshape(1000,1)))
test_in_sort = test_in_sort[test_in_sort[:,-1].argsort()]
test_in_sort[:,-1] = 1
test_out_sort = sorted(test_out)

In [140]:
s = 0
index_test = []
for i in range(len(test_out_sort)):
    if test_out_sort[i] > s:
        index_test.append(i-1)
        s += 1
index_test.append(1000)

In [141]:
index_test

[223, 344, 445, 524, 610, 665, 755, 819, 911, 1000]

In [155]:
from sklearn.neighbors import KNeighborsClassifier as knn

In [157]:
knn_model = knn()
model_knn = knn_model.fit(train_in,train_out)
test_pred_knn = model_knn.predict(test_in)

In [167]:
test_pred_knn.shape

(1000,)

## Compare

In [201]:
w0 = np.random.uniform(-0.3, 0.3, size=(257,10))

train_in_p = preprocessing(train_in)
train_out_dummy = np.zeros(len(train_out)*10).reshape(len(train_out), 10)
train_out_dummy = get_dummies(train_out, train_out_dummy)

w = training(lr, w0, train_in_p, train_out_dummy)

test_out_sort_dummy = np.zeros(Len_test*10).reshape(Len_test, 10)
test_out_sort_dummy = get_dummies(test_out_sort, test_out_sort_dummy)

start = 0

index = 0
for end in index_test:
    each_test_in = test_in_sort[:][start:end]
    each_test_out = test_out_sort[:][start:end]
    each_test_out_sort_dummy = test_out_sort_dummy[:][start:end]

    pred_test = predict(lr, w, each_test_in, each_test_out_sort_dummy)
    pred_test_knn = model_knn.predict(each_test_in[:, 0:-1])
    # pred_test_D = D_classify111(dataset=each_test_in[:, 0:-1])

    acc_test = accuracy_score(each_test_out, pred_test)
    acc_test_knn = accuracy_score(each_test_out, pred_test_knn)
    # acc_test_D = accuracy_score(each_test_out, pred_test_D)
    print("number:",index,"Accuracy test", "single layer perceptron",acc_test, "knn",acc_test_knn, "D_classify")

    index += 1
    start = end

test_out_dummy = np.zeros(Len_test*10).reshape(Len_test, 10)
test_out_dummy = get_dummies(test_out, test_out_dummy)
test_in_p = preprocessing(test_in)


pred_test = predict(lr, w, test_in_p, test_out_dummy)
pred_test_knn = model_knn.predict(test_in)
# pred_test_D = D_classify111(dataset=test_in)

acc_test = accuracy_score(test_out, pred_test)
acc_test_knn = accuracy_score(test_out, pred_test_knn)
# acc_test_D = accuracy_score(test_out, pred_test_D)
print("Accuracy test", "single layer perceptron",acc_test, "knn",acc_test_knn)

629
263
230
218
214
203
number: 0 Accuracy test single layer perceptron 0.9596412556053812 knn 0.9775784753363229 D_classify
number: 1 Accuracy test single layer perceptron 0.9586776859504132 knn 0.9834710743801653 D_classify
number: 2 Accuracy test single layer perceptron 0.8316831683168316 knn 0.8415841584158416 D_classify
number: 3 Accuracy test single layer perceptron 0.8354430379746836 knn 0.8734177215189873 D_classify
number: 4 Accuracy test single layer perceptron 0.872093023255814 knn 0.9186046511627907 D_classify
number: 5 Accuracy test single layer perceptron 0.7090909090909091 knn 0.6545454545454545 D_classify
number: 6 Accuracy test single layer perceptron 0.9444444444444444 knn 0.9333333333333333 D_classify
number: 7 Accuracy test single layer perceptron 0.875 knn 0.875 D_classify
number: 8 Accuracy test single layer perceptron 0.75 knn 0.8586956521739131 D_classify
number: 9 Accuracy test single layer perceptron 0.8876404494382022 knn 0.9325842696629213 D_classify
Accurac